<a href="https://www.kaggle.com/code/romankovalchuk/lora-ua-gec?scriptVersionId=167469943" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!git clone https://github.com/Reennon/ua-gec-lora.git
!cd ua-gec-lora && pip install -r requirements.txt
!pwd && ls -a
# Install additional libs
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
# CD into the project directory
%cd ua-gec-lora
!git pull origin "feature/fine-tuning-research"
!git status

Cloning into 'ua-gec-lora'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 78 (delta 15), reused 73 (delta 12), pack-reused 0
Unpacking objects: 100% (78/78), 68.78 KiB | 2.99 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 24.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 1.7 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 13.6 MB/s 

In [2]:
import torch
from transformers import AutoModelForCausalLM, pipeline, Conversation, AutoTokenizer
from src.packages.utils.parameter_server import ParameterServer
import torch

2024-03-17 13:12:24.000195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 13:12:24.000298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 13:12:24.136119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
print(torch.cuda.is_available())

True


In [4]:
parameter_server = ParameterServer()

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2" #"mistralai/Mistral-7B-Instruct-v0.2" # "bardsai/jaskier-7b-dpo-v6.1" # "microsoft/phi-2" # "bardsai/jaskier-7b-dpo-v6.1"

In [5]:
model = AutoModelForCausalLM.from_pretrained(base_model_name, load_in_4bit=True, device_map="cuda")
model

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 7,243,436,032 || trainable%: 0.023523863432663224


In [8]:
!pip install ua_gec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.0/36.0 MB 42.4 MB/s eta 0:00:00:00:0100:01


In [177]:
from langchain.prompts import PromptTemplate

template = """[INST]Given ("ORIGINAL_TEXT") text with errors, correct them if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language.
Consider following set of possible error types ("ERROR_TYPES"):
{error_types}
Throughout the text, if you would detect error ("ERROR"), fix it according to the structure:
("ERROR") => ("CORRECTION")
Where ("CORRECTION") Specifies how the error should be corrected, without reasoning the change.
The generated text ("FIXED_TEXT") should not contain original errors, or part of these instructions.
Correct only Ukrainian language, and do not correct or translate foreign words, like English.
Keep original information intact, prioritizing the semantics of original text being intact.

ORIGINAL_TEXT: {query}
FIXED_TEXT:
[/INST]"""

it_prompt = PromptTemplate(
    template=template,
    input_variables=['query', 'error_types']
)

In [186]:
!pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [187]:
import nltk
nltk.download('punkt')  # Download the necessary resources for sentence tokenization

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [196]:
from src.packages.constants.error_constants import ErrorConstants
from src.packages.prompts.instruction_tuning_gec_prompts import InstructionTuningGecPrompts
from ua_gec import Corpus
corpus = Corpus(partition="train", annotation_layer="gec-only")
for doc in corpus:
    print("\n---Source starts:---\n")
    print(doc.source[:1000])         # "I likes it."
    print("\n---Source ends:---\n")
    print("\n---Target starts:---\n")
    print(doc.target[:1100])         # "I like it."
    print("\n---Target ends---\n")
    print("\n---Annotation starts:---\n")
    print(str(doc.annotated)[:1200])      # <AnnotatedText("I {likes=>like} it.")
    print(doc.meta.region)    # "Київська"
    print("\n---Annotation ends")
    print("\n---Prompt starts")
    prompt = it_prompt.format_prompt(
        query="".join(sent_tokenize(doc.source)[:6]),
        error_types=ErrorConstants.ERROR_TYPES
    )
    print(prompt)
    break


---Source starts:---

Byte for France або “Мій досвід ведення блогу у Instagram”
Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.

Сьогодні розповім про те як і навіщо мене занесло у Instagram. Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – https://www.instagram.com/yevhenii_kanivets/

Моє бачення Instagram
Колись давно я прочитав статтю, чи просто коментарій – вже не згадаю. Але йшлося там про те, що найпопулярнішою соціальною мережею стане платформа на котрій можно буде лише ділитися світлинами та ставити лайки.

Було це за декілька років до появи усім відомого сервісу. Як же автору вдалося передбачити майбутнє? Дуже просто!

Instagram втілює глибинні бажання кожної людини:

емоційне збудження від перегляду гарних фото (читання – це менш природний процес, ніж споглядання)
нескінченне поглинан

In [11]:
InstructionTuningGecPrompts.PROMPT

PromptTemplate(input_variables=['error_types', 'query'], template='Given ("SOURCE") text with errors, correct them if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language.\nConsider following set of error types ("ERROR_TYPES"):\n{error_types}\nThroughout the text, if you would detect error ("ERROR"), fix it according to the structure:\n{{("ERROR")=>("CORRECTION"):::("ERROR_TYPE")}}\nWhere ("CORRECTION") Specifies how the error should be corrected.\nKeep the original text, only correcting errors in it, without outputting this instructions.\n\nSOURCE: {query}\nFINAL ANSWER:')

In [12]:
!git fetch && git pull

Already up to date.


In [197]:
prompt_len = len(tokenizer.tokenize(prompt.to_string()))
tokenizer_max_len = 768
max_correction_addtional_tokens = 0.1
max_new_tokens = prompt_len + int(prompt_len * 0.1)

print(f"""
Tokenizer max tokens: {tokenizer_max_len}
Prompt len: {prompt_len}
Max token difference because of corrections: {max_correction_addtional_tokens}
Max new tokens len (output without input): {max_new_tokens}
""")


Tokenizer max tokens: 768
Prompt len: 543
Max token difference because of corrections: 0.1
Max new tokens len (output without input): 597



In [190]:
# Fix padding token for Mistral and Phi-2 models
tokenizer.pad_token = "[PAD]"

In [198]:
model_inputs = tokenizer(
    prompt.to_string(), 
    max_length=tokenizer_max_len, 
    padding="max_length", 
    truncation=True, 
    return_tensors="pt"
)
model_inputs

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [199]:
response = peft_model.generate(
    input_ids=model_inputs["input_ids"].to("cuda"),
    attention_mask=model_inputs["attention_mask"].to("cuda"),
    max_new_tokens=max_new_tokens
)
response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[   0,    0,    0,  ..., 4714, 2974,    2]], device='cuda:0')

In [200]:
decoded_outputs = tokenizer.batch_decode(response.detach().cpu().numpy(), skip_special_tokens=True)
text = decoded_outputs[0][len(prompt.to_string())-1:]
text

'] Byte for France або "Мій досвід ведення блогу на Instagram"\n\nОстанні три місяці мого життя видалися надто засипаними подіями та емоціями, але нарешті у мене з\'явилося декілька вільних годин та малої енергії, щоб продовжити серію записів щодо мого досвіду блогерства.Сьогодні я розповім про те, як і для чого мене занесло на Instagram.Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – <https://www.instagram.com/yevhenii_kanivets/>\n\nМоє бачення Instagram\n\nКолись давно я прочитав статтю, чи було це коментарій – вже не згадую.Але йшлося там про те, що найпопулярнішою соціальною мережею стане платформа, на якій можна буде лише розміщувати зображення та ставити лайки.Було це за кілька років до появи усім відомого сервісу.Як автору вдалося передбачити майбутнє?\n\n(Note: The text provided is grammatically correct in Ukrainian language. There are no errors to correct.)'

In [202]:
from difflib import SequenceMatcher


def highlight_changes(text1, text2):
    # Tokenize the texts into words
    words1 = re.findall(r'\w+|[^\w\s]', text1)
    words2 = re.findall(r'\w+|[^\w\s]', text2)


    # Find the unique words present in both texts
    all_words = set(words1 + words2)

    # Initialize a SequenceMatcher object
    matcher = SequenceMatcher(None, words1, words2)

    # Get the differences
    diff = matcher.get_opcodes()

    highlighted_text = []

    for op, start1, end1, start2, end2 in diff:
        if op == 'equal':
            # No change, just append the words as is
            highlighted_text.extend(words1[start1:end1])
        elif op == 'delete':
            # Word(s) removed, highlight with red
            for word in words1[start1:end1]:
                word = '\u0336'.join(word) + '\u0336'
                highlighted_text.append('\033[91m\033[1m' + word + '\033[0m')
        elif op == 'insert':
            # Word(s) added, highlight with green
            for word in words2[start2:end2]:
                highlighted_text.append('\033[92m\033[1m' + word + '\033[0m')
        elif op == 'replace':
            # Word(s) replaced, highlight with yellow
            for word in words2[start2:end2]:
                highlighted_text.append('\033[93m\033[1m' + word + '\033[0m')

    return ' '.join(highlighted_text)

def generate_original_corrected_texts(original_text, corrected_text, highlighted_comparison):
    # Split the original and corrected texts
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    # Initialize empty lists for marked original and corrected texts
    marked_original_text = []
    marked_corrected_text = []

    # Track words from the original text that were removed
    removed_words = set(original_words) - set(corrected_words)

    # Track words from the corrected text that were added
    added_words = set(corrected_words) - set(original_words)

    # Mark removed words in the original text as red
    for word in original_words:
        if word in removed_words:
            marked_original_text.append('\033[91m\033[1m' + word + '\033[0m')
        else:
            marked_original_text.append(word)

    # Mark added words in the corrected text as green
    for word in corrected_words:
        if word in added_words:
            marked_corrected_text.append('\033[92m\033[1m' + word + '\033[0m')
        else:
            marked_corrected_text.append(word)

    return (' '.join(marked_original_text), ' '.join(marked_corrected_text), highlighted_comparison)

text1 = normalize_spaces("".join(sent_tokenize(doc.source)[:6]))
text2 = normalize_spaces(text[2:])

highlighted_text = highlight_changes(text1, text2)

original_text, corrected_text, _ = generate_original_corrected_texts(
    original_text=text1, 
    corrected_text=text2, 
    highlighted_comparison=highlighted_text)

print("Original Text:")
print(original_text)
print()

print("Corrected Text:")
print(corrected_text)
print()

print("Changes comparison:")
print(highlighted_text)


Original Text:
Byte for France або “Мій досвід ведення блогу у Instagram” Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.Сьогодні розповім про те як і навіщо мене занесло у Instagram.Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – https://www.instagram.com/yevhenii_kanivets/ Моє бачення Instagram Колись давно я прочитав статтю, чи просто коментарій – вже не згадаю.Але йшлося там про те, що найпопулярнішою соціальною мережею стане платформа на котрій можно буде лише ділитися світлинами та ставити лайки.Було це за декілька років до появи усім відомого сервісу.Як же автору вдалося передбачити майбутнє?

Corrected Text:
Byte for France або "Мій досвід ведення блогу на Instagram" Останні три місяці мого життя видалися надто засипаними подіями та емоціями, але нарешті у мене з'явилося декілька вільних